<b>WandB Notes</b>

* Code save
* Logging (HTML)
* WandB conifg (run config)
* WandB callback (super?)
* run.save (model summary)



<div align="center">

# 🚀 Spearecode Preprocessing 🚀

</div>

<br>

Welcome to the **Spearecode Preprocessing Notebook**! This notebook will guide you through the necessary preprocessing steps to prepare a toy dataset for Language Model training. We will focus on making the dataset more suitable for training by performing the following steps:

1. 📚 **Loading the dataset**: We'll start by importing the dataset from a file or external source.
2. 📦 **Chunking the text**: The dataset will be divided into smaller chunks or segments, making it easier to process during training.
3. 💬 **Tokenization**: Each chunk of text will be split into individual tokens (words or subwords), which are the basic units for language models.
4. 📊 **Basic Exploratory Data Analysis (EDA)**: We'll analyze the dataset's characteristics, such as token frequency, to gain insights and identify potential issues.

After completing the preprocessing and EDA, the toy dataset will be converted into `TFRecords` format. This efficient binary format is designed for use with TensorFlow and will enable seamless integration with your Language Model training pipeline.

Let's dive in and start preprocessing the dataset! 🎉


<br><br>

<div align="center">

# 🌟 Table of Contents 🌟

</div>

---

0. [**Setup**](#setup)
1. [**Loading the Dataset**](#loading-the-dataset)
2. [**Chunking the Text**](#chunking-the-text)
3. [**Tokenization**](#tokenization)
4. [**Basic Exploratory Data Analysis (EDA)**](#basic-eda)
5. [**Converting to TFRecords**](#converting-to-tfrecords)

---



<br>

<div align="center">

## 🛠️ Setup <a name="setup"></a>

</div>

<br>

In this section, we'll import required libraries and methods from our utilities file. We will also define relevant paths and high level information we may need later. We also run a few basic Tensorflow setup steps to ensure optimal and reproducible runs.

In [3]:
# !pip install -q --upgrade tokenizer-viz
# !pip install -q --upgrade langchain
# !pip install -q --upgrade wandb

# Regular imports (native python and pypi packages)
import os
import sys
import random
import langchain
import numpy as np
import pandas as pd
from glob import glob
import tensorflow as tf
import sentencepiece as spm
from IPython.display import HTML, display
from tokenizer_viz import TokenVisualization
from tqdm.notebook import tqdm; tqdm.pandas()

# Add project root into path so imports work
PROJECT_DIR = os.path.dirname(os.getcwd())
sys.path.insert(0, PROJECT_DIR) 

import wandb
wandb.init(settings=wandb.Settings(code_dir=PROJECT_DIR))

# Our project imports
from spearecode.utils.preprocessing_utils import (
    load_from_txt_file, preprocess_shakespeare, save_to_txt_file, print_check_speare
)
from spearecode.utils.general_utils import (
    tf_xla_jit, tf_set_memory_growth, seed_it_all, flatten_l_o_l, print_ln
)
from spearecode.utils.filtering_utils import (
    save_ds_version, drop_str_from_col_names, pad_truncate_centered,
    get_metadata_df, check_chunks, tokenize, get_n_tokens,
    get_n_lines, get_n_chars
)
from spearecode.utils.tfrecord_utils import write_tfrecords, load_tfrecord_dataset


### DEFINE PATHS --- [PROJECT_DIR="/home/paperspace/home/spearecode"] --- ###
NBS_PATH = os.path.join(PROJECT_DIR, "nbs")
DATA_PATH = os.path.join(PROJECT_DIR, "data")
SS_TEXT_PATH = os.path.join(DATA_PATH, "t8.shakespeare.txt")
PREPROCESSED_FULL_TEXT_PATH = SS_TEXT_PATH.replace(".txt", "_preprocessed.txt")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/paperspace/.netrc


<br>

<div align="center">

## 📚 Loading the Dataset <a name="loading-the-dataset"></a>

</div>

<br>

In this section, we'll import the dataset from a file or external source. The dataset will be read into memory, allowing us to manipulate and process the text as needed throughout the preprocessing steps.


In [36]:
raw_text = load_from_txt_file(SS_TEXT_PATH)
ss_text = preprocess_shakespeare(raw_text)
save_to_txt_file(ss_text, PREPROCESSED_FULL_TEXT_PATH)
print_check_speare(ss_text)


... DATASET INFO:
	NUMBER OF CHARS --> 5,419,872
	NUMBER OF LINES --> 120,696


... FIRST 1000 CHARACTERS:



1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.

2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where al

<br>

<div align="center">

## 📦 Chunking the Text <a name="chunking-the-text"></a>

</div>

<br>

Once the dataset is loaded, we'll divide it into smaller chunks or segments. This step is crucial for making the dataset more similar to code files (which is the type of data we will be using during the other parallel streams).

I implement two simple methods:
1. A basic double newline split **(`\n\n`)** resulting in 6294 chunks
2. Using Langchain RecursiveTextSplitter to chunk to a particular text length
    * This allows us to specify our desired text length and even overlap the chunks.
        * Note we allow for a small amount of overlap and this may cause some leakage... but whatever.
    * **We will use this method for our purposes.**
    


In [37]:
def do_rcts_chunking(text, chunk_size=1280, chunk_overlap=428, length_fn=len):
    """
    Perform Recursive Character Text Splitting (RCTS) chunking on the input text.
    
    Args:
        text (str): The input text to be chunked.
        chunk_size (int): The maximum size of each chunk.
        chunk_overlap (int): The number of overlapping characters between adjacent chunks.
        length_fn (callable, optional): Function to calculate the length of the text. Defaults to len.
    
    Returns:
        list: A list of chunked text segments.
    """
    # Import the RecursiveCharacterTextSplitter from langchain.text_splitter module
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    
    # Instantiate the text splitter with the specified parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=length_fn,
    )
    
    # Split the input text into chunks
    docs = text_splitter.create_documents([text])
    
    # Return the list of chunked text segments
    return [x.page_content for x in docs if len(x.page_content)>1]

def do_basic_chunking(text, chunk_delimeter="\n\n", max_length=1800, min_length=600):
    """
    Perform basic chunking on the input text using the specified delimiter.
    
    Args:
        text (str): The input text to be chunked.
        chunk_delimeter (str, optional): The delimiter used to split the text. Defaults to "\n\n".
    
    Returns:
        list: A list of chunked text segments.
    """
    # Split the input text based on the specified delimiter (ensure no empty chunks by stripping from ends)
    raw_docs = text.strip(chunk_delimeter).split(chunk_delimeter)
    tmp_docs = []
    docs = []
    
    while len(raw_docs)>0:
        doc = raw_docs.pop()
        
        if len(doc)>max_length:
            raw_docs+=doc.split("\n")
        elif len(doc)<min_length:
            tmp_docs.append(doc)
        else:
            docs.append(doc)
            
        if len("\n".join(tmp_docs))>min_length:
            docs.append("\n".join(tmp_docs))
            tmp_docs = []
    if tmp_docs:
        docs.append("\n".join(tmp_docs))
    
    # Return the list of chunked text segments
    return docs

In [38]:
# Feel free to pass non-default kwargs 
#    -- otherwise the rcts chunks will overlap by 64 and be 512 characters long
CHUNK_STYLE = "rcts" # one of ['basic' | 'rcts']
basic_chunks = do_basic_chunking(ss_text)
rcts_chunks = do_rcts_chunking(ss_text)

print("\n... FIRST BASIC CHUNK ...\n")
print(basic_chunks[0])

print("\n... FIRST RCTS CHUNK ...\n")
print(rcts_chunks[0])

print("\n... EXAMPLE RANDOM BASIC CHUNK ...\n")
print(random.sample(basic_chunks, 1)[0])

print("\n... EXAMPLE RANDOM RCTS CHUNK ...\n")
print(random.sample(rcts_chunks, 1)[0])

print("\n... LAST BASIC CHUNK ...\n")
print(basic_chunks[-1])

print("\n... LAST RCTS CHUNK ...\n")
print(rcts_chunks[-1])



... FIRST BASIC CHUNK ...

  'O, that infected moisture of his eye,
  O, that false fire which in his cheek so glowed,
  O, that forced thunder from his heart did fly,
  O, that sad breath his spongy lungs bestowed,
  O, all that borrowed motion, seeming owed,
  Would yet again betray the fore-betrayed,
  And new pervert a reconciled maid.'
  'Thus merely with the garment of a Grace  
  The naked and concealed fiend he covered,
  That th' unexperient gave the tempter place,
  Which, like a cherubin, above them hovered.
  Who, young and simple, would not be so lovered?
  Ay me, I fell, and yet do question make
  What I should do again for such a sake.

... FIRST RCTS CHUNK ...

1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundanc

wandb: Network error (ReadTimeout), entering retry loop.


<br>

<div align="center">

## 💬 Tokenization <a name="tokenization"></a>

</div>

<br>

In this section, we'll tokenize the text, which involves splitting the chunks into individual tokens (words or subwords). Tokenization is an essential step in preprocessing, as it helps the Language Model understand the basic units of the text and learn meaningful patterns.

* We will train our tokenizer on the non-chunked dataset (after basic preprocessing), however, we will 


In [42]:
# Setup model directory if not already setup
MODEL_DIR = os.path.join(os.path.dirname(DATA_PATH), "models")
if not os.path.isdir(MODEL_DIR): os.makedirs(MODEL_DIR, exist_ok=True)

# User defined parameters (matching alphafold and code tokenization standards)
MODEL_PATH = os.path.join(MODEL_DIR, 'spearecode')
USER_DEFINED_SYMBOLS = ["[MASK]", "[NL]"] # mask and newline replacement
VOCAB_SIZE = 8_000
CHAR_COVERAGE = 1.0000

# If using CLS token
PREPEND_CLS = False
if PREPEND_CLS: USER_DEFINED_SYMBOLS.append(["CLS"])


# Tokenizer parameters (and some defaults)
base_tokenizer_kwargs = dict(
    input = PREPROCESSED_FULL_TEXT_PATH,
    vocab_size=VOCAB_SIZE,
    character_coverage=CHAR_COVERAGE,
    pad_id=0, unk_id=1, bos_id=2, eos_id=3,
    remove_extra_whitespaces=False,
    allow_whitespace_only_pieces=True,
    add_dummy_prefix=False,
    user_defined_symbols=USER_DEFINED_SYMBOLS,
    normalization_rule_name="identity",
    num_threads=os.cpu_count(),
    minloglevel=2,
)

unigram_tokenizer_kwargs = base_tokenizer_kwargs.copy()
unigram_tokenizer_kwargs.update(dict(
    model_prefix=MODEL_PATH+"_uni",
    model_type="unigram",
))

bpe_tokenizer_kwargs = base_tokenizer_kwargs.copy()
bpe_tokenizer_kwargs.update(dict(
    model_prefix=MODEL_PATH+"_bpe",
    model_type="bpe",
))

# train_tokenizer(ALL_TXT_PATHS, MODEL_PATH, VOCAB_SIZE, TOKENIZER_STYLE)
spm.SentencePieceTrainer.Train(**unigram_tokenizer_kwargs)
spm.SentencePieceTrainer.Train(**bpe_tokenizer_kwargs)

sp_uni = spm.SentencePieceProcessor()
sp_uni.load(f'{unigram_tokenizer_kwargs["model_prefix"]}.model')
uni_encoder = lambda x: sp_uni.encode(x.replace("\n", "[NL]"))
uni_decoder = lambda x: sp_uni.decode(x).replace("[NL]", "\n")

sp_bpe = spm.SentencePieceProcessor()
sp_bpe.load(f'{bpe_tokenizer_kwargs["model_prefix"]}.model')
bpe_encoder = lambda x: sp_bpe.encode(x.replace("\n", "[NL]"))
bpe_decoder = lambda x: sp_bpe.decode(x).replace("[NL]", "\n")

In [45]:
print("\n... BPE TOKENIZATION:")
bpe_token_viz = TokenVisualization(
    encoder=bpe_encoder,
    decoder=bpe_decoder,
    background_color="#FBFBFB"
)
_ = bpe_token_viz.visualize(rcts_chunks[0], display_inline=True)

print("\n... UNIGRAM TOKENIZATION:")
uni_token_viz = TokenVisualization(
    encoder=uni_encoder,
    decoder=uni_decoder,
    background_color="#FBFBFB"
)
_ = uni_token_viz.visualize(rcts_chunks[0], display_inline=True)


### FOR FUN AND VIZ ###
print("\n... CHAR TOKENIZATION:")
dumb_char_map_s2i = {x:i for i, x in enumerate(set(basic_chunks[0]))}
dumb_char_map_i2s = {v:k for k,v in dumb_char_map_s2i.items()}
char_token_viz = TokenVisualization(
    encoder=lambda x: [dumb_char_map_s2i.get(_x) for _x in x] if type(x)==str else dumb_char_map_s2i.get(x),
    decoder=lambda x: "".join([dumb_char_map_i2s.get(_x) for _x in x]) if type(x)==list else dumb_char_map_i2s.get(x),
    background_color="#FBFBFB"
)
_ = char_token_viz.visualize(basic_chunks[0], display_inline=True)


... BPE TOKENIZATION:



... UNIGRAM TOKENIZATION:



... CHAR TOKENIZATION:


<br>

<div align="center">

## 📊 Basic Exploratory Data Analysis (EDA) <a name="basic-eda"></a>

</div>

<br>

Here, we'll perform a basic EDA on the dataset to gain insights and identify potential issues. This analysis may include examining token frequency, distribution of chunk lengths, and other relevant characteristics. This information can be helpful in understanding the dataset's structure and guiding further preprocessing decisions.

We will utilize the metadata columns we create to create different versions of the dataset:

<br>

<table>
  <thead>
    <tr>
      <th style="text-align: center; font-weight: bold; width: 15%;">Version</th>
      <th style="text-align: center; font-weight: bold; width: 85%;">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="text-align: center;"><strong>v1</strong></td>
        <td>No filtering, no chunks removed. We just generate metadata and (optional) prepend a <b>[CLS]</b> token to every chunk.</td>
    </tr>
    <tr>
      <td style="text-align: center;"><strong>v2</strong></td>
      <td>Split into individual datasets for bpe, unigram within each chunking technique (4 datasets total)<br>Rename columns to not specify tokenization method to allow for more generalization across interaction</td>
    </tr>
    <tr>
      <td style="text-align: center;"><strong>v3</strong></td>
      <td>Drop small chunks<br>Drop really big chunks</td>
    </tr>
    <tr>
      <td style="text-align: center;"><strong>v4</strong></td>
      <td>Evenly pad/truncated tokenized sequences up to reasonable length (close to max length --> 90th percentile?)</td>
    </tr>
  </tbody>
</table>

<br>



In [46]:
# 1. Setup - Create directories if necessary    
DATASET_DIR = os.path.join(DATA_PATH, "datasets")
if not os.path.isdir(DATASET_DIR): os.makedirs(DATASET_DIR, exist_ok=True)
META_DIR = os.path.join(DATASET_DIR, "meta")
if not os.path.isdir(META_DIR): os.makedirs(META_DIR, exist_ok=True)


# 2. Dataframe and metadata creation
#       - Instantiate
#       - Create metadata columns
#       - Create metadata dataframe (optional)
basic_chunk_df = pd.DataFrame({"content":basic_chunks})
rcts_chunk_df = pd.DataFrame({"content":rcts_chunks})

for _df in [basic_chunk_df, rcts_chunk_df]:
    if PREPEND_CLS: _df["content"] = "[CLS] "+_df["content"]
    _df["uni_token_content"] = _df["content"].progress_apply(lambda x: tokenize(x, uni_encoder))
    _df["bpe_token_content"] = _df["content"].progress_apply(lambda x: tokenize(x, bpe_encoder))
    _df["n_uni_tokens"] = _df["uni_token_content"].apply(get_n_tokens)
    _df["n_bpe_tokens"] = _df["bpe_token_content"].apply(get_n_tokens)
    _df["n_chars"] = _df["content"].apply(get_n_chars)
    _df["n_lines"] = _df["content"].apply(get_n_lines)
    _df["valid_uni_chunk"] = _df["n_uni_tokens"].apply(check_chunks)
    _df["valid_bpe_chunk"] = _df["n_bpe_tokens"].apply(check_chunks)

basic_chunk_df_meta = get_metadata_df(basic_chunk_df)
rcts_chunk_df_meta = get_metadata_df(rcts_chunk_df)


# 3. Versioning
    
######################################## v1 ########################################
# Save the previously created datasets along with the manually created metadata
####################################################################################
save_ds_version(rcts_chunk_df, "rcts", version_str="v1", meta_dir=META_DIR, ds_dir=DATASET_DIR, meta_df=rcts_chunk_df_meta)
save_ds_version(basic_chunk_df, "basic", version_str="v1", meta_dir=META_DIR, ds_dir=DATASET_DIR, meta_df=basic_chunk_df_meta)
####################################################################################

######################################## v2 ########################################
# Split bpe and unigram into their own dataframes (meta is generated automatically)
####################################################################################
rcts_uni_chunk_df = rcts_chunk_df.copy().drop(columns=[_c for _c in rcts_chunk_df.columns if "bpe" in _c])
basic_uni_chunk_df = basic_chunk_df.copy().drop(columns=[_c for _c in basic_chunk_df.columns if "bpe" in _c])
rcts_bpe_chunk_df = rcts_chunk_df.copy().drop(columns=[_c for _c in rcts_chunk_df.columns if "uni" in _c])
basic_bpe_chunk_df = basic_chunk_df.copy().drop(columns=[_c for _c in basic_chunk_df.columns if "uni" in _c])

# Rename columns
rcts_uni_chunk_df = drop_str_from_col_names(rcts_uni_chunk_df, "uni")
rcts_bpe_chunk_df = drop_str_from_col_names(rcts_bpe_chunk_df, "bpe")
basic_uni_chunk_df = drop_str_from_col_names(basic_uni_chunk_df, "uni")
basic_bpe_chunk_df = drop_str_from_col_names(basic_bpe_chunk_df, "bpe")

save_ds_version(rcts_uni_chunk_df, "rcts_uni", version_str="v2", meta_dir=META_DIR, ds_dir=DATASET_DIR)
save_ds_version(rcts_bpe_chunk_df, "rcts_bpe", version_str="v2", meta_dir=META_DIR, ds_dir=DATASET_DIR)
save_ds_version(basic_uni_chunk_df, "basic_uni", version_str="v2", meta_dir=META_DIR, ds_dir=DATASET_DIR)
save_ds_version(basic_bpe_chunk_df, "basic_bpe", version_str="v2", meta_dir=META_DIR, ds_dir=DATASET_DIR)
####################################################################################

######################################## v3 ########################################
# Filtering (big and little get dropped)
####################################################################################

# Filter and drop valid chunk col
for _df in [rcts_uni_chunk_df, rcts_bpe_chunk_df, basic_uni_chunk_df, basic_bpe_chunk_df]:
    _df = _df[_df.valid_chunk].drop(columns=["valid_chunk"]).reset_index(drop=True)

# Save
save_ds_version(rcts_uni_chunk_df, "rcts_uni", version_str="v3", meta_dir=META_DIR, ds_dir=DATASET_DIR)
save_ds_version(rcts_bpe_chunk_df, "rcts_bpe", version_str="v3", meta_dir=META_DIR, ds_dir=DATASET_DIR)
save_ds_version(basic_uni_chunk_df, "basic_uni", version_str="v3", meta_dir=META_DIR, ds_dir=DATASET_DIR)
save_ds_version(basic_bpe_chunk_df, "basic_bpe", version_str="v3", meta_dir=META_DIR, ds_dir=DATASET_DIR)
####################################################################################

# ######################################## v4 ########################################
# # Padding and truncation --> basic upper limit of 384 (assuming context lengths of 64-128)
# ####################################################################################
FIXED_CHUNK_SIZE = 384

for _df in [rcts_uni_chunk_df, rcts_bpe_chunk_df, basic_uni_chunk_df, basic_bpe_chunk_df]:
    _df["token_content"] = _df["token_content"].apply(lambda x: pad_truncate_centered(x, FIXED_CHUNK_SIZE))

save_ds_version(rcts_uni_chunk_df, "rcts_uni", version_str="v4", meta_dir=META_DIR, ds_dir=DATASET_DIR)
save_ds_version(rcts_bpe_chunk_df, "rcts_bpe", version_str="v4", meta_dir=META_DIR, ds_dir=DATASET_DIR)
save_ds_version(basic_uni_chunk_df, "basic_uni", version_str="v4", meta_dir=META_DIR, ds_dir=DATASET_DIR)
save_ds_version(basic_bpe_chunk_df, "basic_bpe", version_str="v4", meta_dir=META_DIR, ds_dir=DATASET_DIR)
# ####################################################################################

  0%|          | 0/7770 [00:00<?, ?it/s]

  0%|          | 0/7770 [00:00<?, ?it/s]

  0%|          | 0/6986 [00:00<?, ?it/s]

  0%|          | 0/6986 [00:00<?, ?it/s]

<br>

<div align="center">

## 💾 Converting to TFRecords <a name="converting-to-tfrecords"></a>

</div>

<br>

Finally, after completing the preprocessing steps and EDA, we'll convert the toy dataset into the `TFRecords` format. This efficient binary format is designed for use with TensorFlow and will enable seamless integration with your Language Model training pipeline.



In [50]:
# Define tfrecord creation constants
TFRECORD_DIR = os.path.join(DATASET_DIR, "tfrecords")
N_PER = 100 # artificially low to replicate tfrecord amounts expected
VERSION_TO_USE = "v4"

for _df, _suffix in zip([rcts_uni_chunk_df, rcts_bpe_chunk_df, basic_uni_chunk_df, basic_bpe_chunk_df], 
                        ["rcts_uni", "rcts_bpe", "basic_uni", "basic_bpe"]):
    # Create the respective tfrecords
    write_tfrecords(
        ds=_df["token_content"],  n_ex=len(_df),  
        output_suffix=_suffix,  version_str=VERSION_TO_USE, 
        n_ex_per_rec=N_PER, out_dir=TFRECORD_DIR, 
    )

Writing TFRecords:   0%|                                                                  | 0/70 [00:00<?, ?it/s]


... Writing TFRecord 1 of 70 (100 per TFRecord)...




100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 15392.51it/s]



... Writing TFRecord 2 of 70 (100 per TFRecord)...




100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 13807.50it/s]


...


Writing TFRecords:  97%|██████████████████████████████████████████████████████▌ | 76/78 [00:00<00:00, 100.85it/s]


... Writing TFRecord 77 of 78 (100 per TFRecord)...




100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 14438.72it/s]



... Writing TFRecord 78 of 78 (100 per TFRecord)...




Writing TFRecords: 100%|████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 101.12it/s]


<br>

**Check dataset**

In [51]:
DEMO_DS_CHUNK_STYLE, DEMO_DS_TOK_STYLE, DEMO_DS_VERSION = "rcts", "bpe", "v4"
DEMO_TFREC_PATHS = sorted(glob(os.path.join(
    TFRECORD_DIR, f"{DEMO_DS_CHUNK_STYLE}_{DEMO_DS_TOK_STYLE}_{DEMO_DS_VERSION}", "*.tfrec"
)))

# Get respective tooling
demo_ds = load_tfrecord_dataset(DEMO_TFREC_PATHS)
viz_tool = bpe_token_viz if DEMO_DS_TOK_STYLE=="bpe" else uni_token_viz

# Check and compare
print("\n... FROM TFRECORD ...\n")
display(HTML(viz_tool.visualize(bpe_decoder(next(iter(demo_ds)).numpy().tolist()))))

# TODO make modular... not important
print("\n... FROM PANDAS DATAFRAME ...\n")
display(HTML(viz_tool.visualize(bpe_decoder(rcts_bpe_chunk_df["token_content"][0]))))


... FROM TFRECORD ...




... FROM PANDAS DATAFRAME ...



<br>

<b>Send Some wandb Stuff</b>

In [66]:
from wandb import Table

rcts_bpe_wandb_table  = Table(dataframe=get_metadata_df(rcts_bpe_chunk_df))
rcts_uni_wandb_table  = Table(dataframe=get_metadata_df(rcts_uni_chunk_df))
basic_bpe_wandb_table = Table(dataframe=get_metadata_df(basic_bpe_chunk_df))
basic_uni_wandb_table = Table(dataframe=get_metadata_df(basic_uni_chunk_df))

wandb.log({
    "rcts_bpe": rcts_bpe_wandb_table,
    "rcts_uni": rcts_uni_wandb_table,
    "basic_bpe": basic_bpe_wandb_table,
    "basic_uni": basic_uni_wandb_table,
})
wandb.finish()